In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import dataloader as dl

In [ ]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10
USE_CUDA = torch.cuda.is_available()
#DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)

In [ ]:
train_loader=dl.load_trainset()
test_loader=dl.load_testset()

In [ ]:
class Discriminator(nn.Module):
    #def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
    def __init__(self, n_layers, hidden_dim, input_shape, n_classes, dropout_p=0.2):
        super(Discriminator, self).__init__()
        print("Building Basic Discriminator model...")
        self.n_layers = n_layers 
        #self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(input_shape, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, n_classes)
        self.out = nn.Softmax()
    def forward(self, x):
        #x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        c_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.lstm(x, (h_0, c_0))  # [i, b, h]
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.fc(h_t)  # [b, h] -> [b, o]
        logit = self.out(logit)
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [ ]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [1]:
# Generator Code
#nc - number of color channels in the input images. For color images this is 3
#nz - length of latent vector (i.e. size of generator input)
#ngf - relates to the depth of feature maps carried through the generator (Size of feature maps in generator)
#ndf - sets the depth of feature maps propagated through the discriminator (Size of feature maps in discriminator)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

nc=1 #channel이랄 것이 딱히 없음


class DCGANGenerator(nn.Module):
    def __init__(self):
        super(DCGANGenerator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),a
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )
    
    
    def forward(self, input):
        return self.main(input)

SyntaxError: invalid syntax (<ipython-input-1-0cbfba8bdaff>, line 25)

In [ ]:
class CNNGenerator(nn.Module):
    def __init__(self, n_layers, hidden_dim, input_shape, dropout_p=0.2):
        super(CNNGenerator, self).__init()
        print("Building CNN Generator")

In [ ]:
class Generator(nn.Module):
    def __init__(self, n_layers, hidden_dim, input_shape, dropout_p=0.2):
        super(Generator, self).__init__()
        print("Building Basic generator model...")
        self.n_layers = n_layers 
        #self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(input_shape, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        #self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        #x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        c_0 = self._init_state(batch_size=x.size(0))
        x, (h, c) = self.lstm(x, (h_0, c_0))  # [i, b, h]
        
        return x
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [ ]:
D = Discriminator(1, 256, 75, 2, 0.2).to(DEVICE)
G = Generator(1, 75, 1, 0.2).to(DEVICE)

d_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

In [ ]:
criterion=nn.BCELoss()

In [ ]:
input_data=torch.randn(30,1)
input_data=torch.tensor(np.array(input_data)).unsqueeze(0).to(DEVICE)

In [ ]:
total_step = len(data_loader)
for epoch in range(EPOCHS):
    for i, data in enumerate(data_loader):
        x = np.array(data[0][0])
        
        X=[]
        for i in range(10):
            X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        
        X=torch.tensor(np.array(X)).unsqueeze(0).to(DEVICE)
        
        # '진짜'와 '가짜' 레이블 생성
        real_labels = torch.tensor([[0,1]]).to(torch.float32).to(DEVICE)
        fake_labels = torch.tensor([[1,0]]).to(torch.float32).to(DEVICE)
        
        # 판별자가 진짜 이미지를 진짜로 인식하는 오차를 예산
        outputs = D(X)
        
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        #print(real_score)
        # 무작위 텐서로 가짜 이미지 생성
        z = torch.randn(30,1)
        z = torch.tensor(np.array(z)).unsqueeze(0).to(DEVICE)
        fake_images = G(z)
        
        # 판별자가 가짜 이미지를 가짜로 인식하는 오차를 계산
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 진짜와 가짜 이미지를 갖고 낸 오차를 더해서 판별자의 오차 계산
        d_loss = d_loss_real + d_loss_fake

        # 역전파 알고리즘으로 판별자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 생성자가 판별자를 속였는지에 대한 오차를 계산
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        
        # 역전파 알고리즘으로 생성자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    # 학습 진행 알아보기
    print('Epoch [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
          .format(epoch, EPOCHS, d_loss.item(), g_loss.item(), 
                  real_score.mean().item(), fake_score.mean().item()))
    if(epoch%25==0):
        torch.save(G.state_dict(), './snapshot/gan_'+str(epoch)+'.pt')